### Import Package and set path

In [1]:
import pandas as pd
from pathlib import Path
import glob
import datetime

import os
import numpy as np
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from datetime import date
import math
import shutil

In [2]:
# home and time
home = os.path.expanduser("~")
todaystr = date.today().strftime('%Y-%m-%d')

# set up concat directories
targetFolder = os.path.join(home, 'HP Inc','GPSTW SOP - 2021 日新','Project team','Upload folder ( for buyer update )')
FD_folder = os.path.join(targetFolder, "FD_today")
shortage_folder = os.path.join(targetFolder ,"shortage_today")
PNbasedDetail_folder = os.path.join(targetFolder ,"PNbasedDetail_today")

### Merge and clean FD

#### Merge

In [3]:
# concat FD
FD_files = []
for FD_file in glob.glob(str(os.path.join(FD_folder,"*.xlsx"))):
    print(FD_file)
    FD = pd.read_excel(FD_file)
    FD_files.append(FD)
FD_all = pd.concat(FD_files)

C:\Users\lulo\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\FD_today\20230104_BATTERY_CEI_ChenVivian(GPS)_FD.xlsx
C:\Users\lulo\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\FD_today\20230104_BATTERY_CQQCI_ChenVivian(GPS)_FD.xlsx
C:\Users\lulo\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\FD_today\20230105_BATTERY_CEI_ChenVivian(GPS)_FD.xlsx
C:\Users\lulo\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\FD_today\20230105_BATTERY_CQQCI_ChenVivian(GPS)_FD.xlsx
C:\Users\lulo\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\FD_today\20230106_BATTERY_CEI_ChenVivian(GPS)_FD.xlsx


In [4]:
# remember inplace = True
FD_all['len_FV'] = FD_all['FV'].str.len()
FD_all['len_platform'] = FD_all['Platform'].str.len()
FD_all = FD_all.sort_values( ['len_FV','len_platform'] , ascending = [False,False] )
FD_all.reset_index( drop = True, inplace = True )

#### Move row with max len to the top row

In [5]:
# Index where B is longest
FD_idmax = FD_all['len_platform'].max()
df_max = FD_all.loc[FD_all['len_platform'] == FD_idmax]
df_max_to_add_FD = df_max.head(1)

In [6]:
# isin用在list
# drop the max len row
FD_all = FD_all.drop([FD_all.index[df_max_to_add_FD.index.values[0]]]).reset_index( drop = True )

In [7]:
# concat and put the max len row on the top
FD_concat = [df_max_to_add_FD,FD_all]
FD_output = pd.concat(FD_concat)
FD_output.reset_index( drop = True , inplace = True)

In [8]:
# cut more than 500
FD_output['Platform'] = FD_output['Platform'].apply(lambda x: x[:450] if len(x) > 500 else x)
FD_output = FD_output.drop( columns = ['len_FV','len_platform'])
FD_output['Item'] = FD_output['Item'].astype(str)

### Merge and clean Shortage

#### Merge

In [9]:
# concat shortage
shortage_files = []
for shortage_file in glob.glob(str(os.path.join(shortage_folder,"*.xlsx"))):
    print(shortage_file)
    shortage = pd.read_excel(shortage_file)
    shortage_files.append(shortage)
shortage_all = pd.concat(shortage_files)

C:\Users\lulo\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\shortage_today\20230104_BATTERY_CEI_ChenVivian(GPS)_Shortage.xlsx
C:\Users\lulo\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\shortage_today\20230104_BATTERY_CQQCI_ChenVivian(GPS)_Shortage.xlsx
C:\Users\lulo\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\shortage_today\20230105_BATTERY_CEI_ChenVivian(GPS)_Shortage.xlsx
C:\Users\lulo\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\shortage_today\20230105_BATTERY_CQQCI_ChenVivian(GPS)_Shortage.xlsx
C:\Users\lulo\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\shortage_today\20230106_BATTERY_CEI_ChenVivian(GPS)_Shortage.xlsx


In [10]:
# create value to sort 
shortage_all['len_FV'] = shortage_all['FV'].str.len()
shortage_all['len_platform'] = shortage_all['Platform'].str.len()
shortage_all = shortage_all.sort_values(['len_FV','len_platform'] , ascending=[False,False])
shortage_all.reset_index( drop = True, inplace = True )

In [11]:
try:
    shortage_all['HP_PN'] = shortage_all['HP_PN'].apply(lambda x: x[:128] if len(x) > 128 else x)
except:    
    pass

#### Move row with max len to the top row

In [12]:
# find the max length 
shortage_idmax = shortage_all['len_platform'].max()
shortage_max = shortage_all.loc[shortage_all['len_platform'] == shortage_idmax]
df_max_to_add_shortage = shortage_max.head(1)
df_max_to_add_shortage

,ODM,Item,Commodity,FV,Platform,HP_PN,P1,Net P2,Net P3,Total Shortage Qty,BT shortage,Working on upside,ReportDate,last FD date,BuyerName,len_FV,len_platform
0,CEI,BATT74,BATTERY,PG03052XL PL,Cyprus 1C19/Milos 1C19/Cyprus 20C1/Milos 20C1/...,"L48430-2C1,L48430-2B1,L48430-AC2,L48430-2C2,L4...",1217,0,0,1217,0,1217,2023-01-04,2023-01-11,ChenVivian(GPS),12,57


In [13]:
# drop the max len row
shortage_all = shortage_all.drop([shortage_all.index[df_max_to_add_shortage.index.values[0]]]).reset_index( drop = True )

In [14]:
# concat and put the max len row on the top
shortage_concat = [ df_max_to_add_shortage , shortage_all ]
shortage_output = pd.concat(shortage_concat)
shortage_output.reset_index( drop = True , inplace = True)

In [15]:
# cut more than 500
shortage_output['Platform'] = shortage_output['Platform'].apply(lambda x: x[:450] if len(x) > 500 else x)
shortage_output = shortage_output.drop( columns= ['len_FV','len_platform'])
shortage_output['Item'] = shortage_output['Item'].astype(str)

### Merge and clean PNbasedDetail

#### Merge

In [16]:
# concat PNbasedDetail
PNbasedDetail_files = []
for PNbasedDetail_file in glob.glob(str(os.path.join(PNbasedDetail_folder,"*.xlsx"))):
    print(PNbasedDetail_file)
    PNbasedDetail = pd.read_excel(PNbasedDetail_file)
    PNbasedDetail_files.append(PNbasedDetail)
PNbasedDetail_all = pd.concat(PNbasedDetail_files)

C:\Users\lulo\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\PNbasedDetail_today\20230104_BATTERY_CEI_ChenVivian(GPS)_PNbasedDetail.xlsx
C:\Users\lulo\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\PNbasedDetail_today\20230104_BATTERY_CQQCI_ChenVivian(GPS)_PNbasedDetail.xlsx
C:\Users\lulo\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\PNbasedDetail_today\20230105_BATTERY_CEI_ChenVivian(GPS)_PNbasedDetail.xlsx
C:\Users\lulo\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\PNbasedDetail_today\20230105_BATTERY_CQQCI_ChenVivian(GPS)_PNbasedDetail.xlsx
C:\Users\lulo\HP Inc\GPSTW SOP - 2021 日新\Project team\Upload folder ( for buyer update )\PNbasedDetail_today\20230106_BATTERY_CEI_ChenVivian(GPS)_PNbasedDetail.xlsx


In [17]:
# create list to sort 
character_limit_list = ['GPS Remark','ODM use column1','ODM use column2','ODM use column3','ODM use column4','ODM use column5']
sort_list = []
asc_list = []
for _ in character_limit_list:

    try:
        PNbasedDetail_all[str('len_' + _)] = PNbasedDetail_all[_].str.len()
        sort_list.append(str('len_' + _))
        asc_list.append(True)
    except:    
        pass

#### Move row with max len to the top row

In [18]:
# sort value
PNbasedDetail_all = PNbasedDetail_all.sort_values( sort_list , ascending = asc_list )
PNbasedDetail_all.reset_index( drop=True , inplace=True ) 

In [19]:
# find the max length & concat
PNbasedDetail_max_files = []

# a for loop to calculate all the len max
for i in range( 1 , len(sort_list) ):
    PNbasedDetail_idmax = PNbasedDetail_all [ sort_list[i] ].max()
    PNbasedDetail_max = PNbasedDetail_all.loc[ PNbasedDetail_all[ sort_list[i] ] == PNbasedDetail_idmax ]
    PNbasedDetail_max_files.append( PNbasedDetail_max.head(1) )
    df_max_to_add_PNbasedDetail_temp = pd.concat( PNbasedDetail_max_files )

# sometimes with duplicates 
df_max_to_add_PNbasedDetail = df_max_to_add_PNbasedDetail_temp.drop_duplicates()

In [20]:
# check point
df_max_to_add_PNbasedDetail

,ODM,Item,Commodity,HP PN,GPS Remark,852 stock,852 stock change,Over pull qty,ODM use column1,ODM use column2,ODM use column3,ODM use column4,ODM use column5,ReportDate,BuyerName,len_ODM use column1,len_ODM use column2
0,CQQCI,1.0,BATTERY,L75253-1C1,NaN,0,0,0,0GB/0GI,GH02047PL-Total,NaN,NaN,NaN,2023-01-04,ChenVivian(GPS),7.0,15.0


In [21]:
# drop the max len row
for i in range( 0, len(df_max_to_add_PNbasedDetail.index.values)):
    PNbasedDetail_all = PNbasedDetail_all.drop([PNbasedDetail_all.index[df_max_to_add_PNbasedDetail.index.values[i]]])

In [22]:
# concat and put on the top
PNbasedDetail_concat_list = [ df_max_to_add_PNbasedDetail , PNbasedDetail_all ]
PNbasedDetail_output = pd.concat(PNbasedDetail_concat_list).reset_index( drop = True )
PNbasedDetail_output

,ODM,Item,Commodity,HP PN,GPS Remark,852 stock,852 stock change,Over pull qty,ODM use column1,ODM use column2,ODM use column3,ODM use column4,ODM use column5,ReportDate,BuyerName,len_ODM use column1,len_ODM use column2
0,CQQCI,1.0,BATTERY,L75253-1C1,NaN,0,0,0,0GB/0GI,GH02047PL-Total,NaN,NaN,NaN,2023-01-04,ChenVivian(GPS),7.0,15.0
1,CQQCI,1.0,BATTERY,L75253-1C1,NaN,0,0,0,0GB/0GI,GH02047PL-Total,NaN,NaN,NaN,2023-01-05,ChenVivian(GPS),7.0,15.0
2,CEI,BATT74,BATTERY,L48430-AC2,NaN,3874,0,0,NaN,NaN,NaN,NaN,NaN,2023-01-04,ChenVivian(GPS),NaN,NaN
3,CEI,BATT74,BATTERY,L48430-AC2,NaN,3874,0,0,NaN,NaN,NaN,NaN,NaN,2023-01-05,ChenVivian(GPS),NaN,NaN
4,CEI,BATT74,BATTERY,L48430-AC2,NaN,3692,0,0,NaN,NaN,NaN,NaN,NaN,2023-01-06,ChenVivian(GPS),NaN,NaN


In [23]:
# cut more than 500
for _ in character_limit_list:
    try:
        PNbasedDetail_output[_] = PNbasedDetail_output[_].apply(lambda x: x[:450] if len(x) > 500 else x)
    except:    
        pass


In [24]:
# final step, drop calculate step and output
PNbasedDetail_output = PNbasedDetail_output.drop( columns = sort_list )
PNbasedDetail_output['Item'] = PNbasedDetail_output['Item'].astype(str)
PNbasedDetail_output

,ODM,Item,Commodity,HP PN,GPS Remark,852 stock,852 stock change,Over pull qty,ODM use column1,ODM use column2,ODM use column3,ODM use column4,ODM use column5,ReportDate,BuyerName
0,CQQCI,1.0,BATTERY,L75253-1C1,NaN,0,0,0,0GB/0GI,GH02047PL-Total,NaN,NaN,NaN,2023-01-04,ChenVivian(GPS)
1,CQQCI,1.0,BATTERY,L75253-1C1,NaN,0,0,0,0GB/0GI,GH02047PL-Total,NaN,NaN,NaN,2023-01-05,ChenVivian(GPS)
2,CEI,BATT74,BATTERY,L48430-AC2,NaN,3874,0,0,NaN,NaN,NaN,NaN,NaN,2023-01-04,ChenVivian(GPS)
3,CEI,BATT74,BATTERY,L48430-AC2,NaN,3874,0,0,NaN,NaN,NaN,NaN,NaN,2023-01-05,ChenVivian(GPS)
4,CEI,BATT74,BATTERY,L48430-AC2,NaN,3692,0,0,NaN,NaN,NaN,NaN,NaN,2023-01-06,ChenVivian(GPS)


### Output concated FD, Shortage, and PNbasedDetail files

In [25]:
# apache airflow to upload SQL ( currently to desktop )
FD_output.to_excel(os.path.join(home, 'Desktop', 'FD_all.xlsx'), index=False)
shortage_output.to_excel(os.path.join(home, 'Desktop', 'Shortage_all.xlsx'), index=False)
PNbasedDetail_output.to_excel(os.path.join(home, 'Desktop', 'PNbasedDetail_all.xlsx'), index=False)

### Move file to archive

In [ ]:
FD_folder = os.path.join(targetFolder, "FD_today")
FD_archive_folder = os.path.join(targetFolder, 'FD_Archive_After_1025')

for f in os.listdir(FD_folder):
    shutil.move(os.path.join(FD_folder, f), os.path.join(FD_archive_folder, f))
    
shortage_folder = os.path.join(targetFolder ,"shortage_today")
shortage_archive_folder = os.path.join(targetFolder ,"Shortage_Archive_After_1025")

for f in os.listdir(shortage_folder):
    shutil.move(os.path.join(shortage_folder, f), os.path.join(shortage_archive_folder, f))

PNbasedDetail_folder = os.path.join(targetFolder ,"PNbasedDetail_today")
PNbasedDetail_archive_folder = os.path.join(targetFolder ,"PNbasedDetail_Archive_After_1025")

for f in os.listdir(PNbasedDetail_folder):
    shutil.move(os.path.join(PNbasedDetail_folder, f), os.path.join(PNbasedDetail_archive_folder, f))
